# Storing of `Text` objects with syntax layer in a PostgreSQL database

This tutorial demonstrates how to store and query EstNLTK `Text` objects with syntax layer in a PostgreSQL database.

In [1]:
from estnltk import Text, logger
from estnltk.taggers import VabamorfTagger, WordTagger
from estnltk.storage.postgres import PostgresStorage, delete_schema
from estnltk.storage.postgres import LayerQuery, WhereClause

## Access to the database

In [2]:
storage = PostgresStorage(host=None,
                          port=None,
                          dbname='test_db',
                          user=None,
                          password=None,
                          pgpass_file='~/.pgpass',
                          schema='my_schema',
                          role=None,
                          temporary=False,
                          create_schema_if_missing=True)

INFO:storage.py:55: connecting to host: 'localhost', port: '5432', dbname: 'test_db', user: 'postgres'
INFO:storage.py:76: new schema 'my_schema' created
INFO:storage.py:106: schema: 'my_schema', temporary: False, role: 'postgres'


In [3]:
collection = storage.add_collection('my_collection')
collection

INFO:storage.py:211: new empty collection 'my_collection' created


## Add texts

Read a text from a conll file.

In [4]:
from estnltk.converters.conll.conll_importer import conll_to_text

text = conll_to_text(file='example.conll', syntax_layer='malt_1')
text

Text(text='Milliseks kujuneb Riigikassa ja Ühispanga vahekord ? Minu arvates on Eesti pangandus tehnoloogiliselt maailma tasemel . Eesti riik on veel suhteliselt laisaks kliendiks ( panganduses on kasutusel termin " laisk raha " , see tähendab ebaratsionaalne hoiustajale , kuid väga hea pangale ) . Valitsused üldiselt ongi nn laisaks kliendiks . Ilmselt need vahekorrad normaliseeruvad , selles mõttes , et optimeerimise käigus riik kui panga hoiustaja hakkab ratsionaalsemalt käituma . Kas kindlustus- , väärtpaberi- ja pangainspektsioon ühendatakse ? Teoreetiliselt on see ehk vale , aga praktiliselt ratsionaalne .')

Split by sentences

In [5]:
from estnltk_core.layer_operations import split_by_sentences

texts = split_by_sentences(text, layers_to_keep=['words', 'malt_1'])

texts[0]

Text(text='Milliseks kujuneb Riigikassa ja Ühispanga vahekord ?')

Note the value of `dict_converter_module` attribute.

In [6]:
texts[0].malt_1.serialisation_module

'syntax_v0'

In [7]:
with collection.insert() as collection_insert:
    for text in texts:
        collection_insert(text)

INFO:collection_text_object_inserter.py:102: inserted 7 texts into the collection 'my_collection'


In [8]:
collection

,layer_type,attributes,ambiguous,sparse,parent,enveloping,meta
words,attached,(),True,False,None,None,[]
malt_1,attached,"(id, lemma, upostag, xpostag, feats, head, deprel, deps, misc, parent_span, children)",False,False,None,None,[]


The layers inserted with the `Text` objects are stored in the same database table with the `Text` object and are called **attached** layers.

### Iterate collection

In [9]:
len(collection)

7

In [10]:
text = collection[0]
text

Text(text='Milliseks kujuneb Riigikassa ja Ühispanga vahekord ?')

In [11]:
text.malt_1

Layer(name='malt_1', attributes=('id', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc', 'parent_span', 'children'), spans=SL[Span('Milliseks', [{'id': 1, 'lemma': 'milline', 'upostag': 'P', 'xpostag': 'P', 'feats': {'sg': '', 'tr': '', 'rel': ''}, 'head': 2, 'deprel': '@ADVL', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('kujuneb', [{'id': 2, 'lemma': 'kujune', 'upostag': 'V', 'xpostag': 'V', 'feats': {'sg': '', 'ps3': '', 'pres': '', 'indic': ''}, 'head': 0, 'deprel': 'ROOT', 'deps': None, 'misc': None, 'parent_span': None, 'children': <class 'tuple'>}]),
Span('Riigikassa', [{'id': 3, 'lemma': 'Riigi_kassa', 'upostag': 'S', 'xpostag': 'H', 'feats': {'sg': '', 'nom': ''}, 'head': 2, 'deprel': '@SUBJ', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('ja', [{'id': 4, 'lemma': 'ja', 'upostag': 'J', 'xpostag': 'Jc', 'feats': None, 'head': 6, 'deprel': '@J', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('Ühispanga', [{'id': 5, 'lemma': 'ühis_pank', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': '', 'gen': ''}, 'head': 6, 'deprel': '@NN>', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('vahekord', [{'id': 6, 'lemma': 'vahe_kord', 'upostag': 'S', 'xpostag': 'S', 'feats': {'sg': '', 'nom': ''}, 'head': 2, 'deprel': '@SUBJ', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('?', [{'id': 7, 'lemma': '?', 'upostag': 'Z', 'xpostag': 'Z', 'feats': {'Int': ''}, 'head': 6, 'deprel': '@Punc', 'deps': None, 'misc': None, 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}])])

In [12]:
storage.delete_collection( collection.name )

In [13]:
delete_schema(storage)
storage.close()